<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto</font></h1>

In this assignment, you will be required to explore, segment, and cluster the neighborhoods in the city of Toronto. However, unlike New York, the neighborhood data is not readily available on the internet. What is interesting about the field of data science is that each project can be challenging in its unique way, so you need to learn to be agile and refine the skill to learn new libraries and tools quickly depending on the project.

For the Toronto neighborhood data, a Wikipedia page exists that has all the information we need to explore and cluster the neighborhoods in Toronto. You will be required to scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas dataframe so that it is in a structured format like the New York dataset.

Once the data is in a structured format, you can replicate the analysis that we did to the New York City dataset to explore and cluster the neighborhoods in the city of Toronto.

In [45]:
import pandas as pd 
import numpy as np
import pixiedust

## Data Preparation

In [60]:
# Using read_html from pandas module
# Picking the first element as it is the table we need
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]

In [47]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [48]:
df.shape

(288, 3)

In [61]:
# Renaming columns
df.columns = ['PostalCode', 'Borough', 'Neighborhood']
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Only process the cells that have an assigned borough. Ignore cells with a borough that is **Not assigned**.

In [50]:
# Checking Borough with 'Not assigned' before processing rows
df['Borough'].value_counts()

Not assigned        77
Etobicoke           45
North York          38
Scarborough         37
Downtown Toronto    37
Central Toronto     17
West Toronto        13
York                 9
East Toronto         7
East York            6
Queen's Park         1
Mississauga          1
Name: Borough, dtype: int64

In [62]:
# Dropping cells with a borough that is Not assigned.
df.replace("Not assigned", np.nan, inplace=True)
df.dropna(axis=0, subset=['Borough'], inplace=True)

In [63]:
df['Borough'].value_counts()

Etobicoke           45
North York          38
Scarborough         37
Downtown Toronto    37
Central Toronto     17
West Toronto        13
York                 9
East Toronto         7
East York            6
Queen's Park         1
Mississauga          1
Name: Borough, dtype: int64

More than one neighborhood can exist in one postal code area. For example, you will notice that **M5A** is listed twice and has two neighborhoods: **Harbourfront** and **Regent Park**. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in **row 11** in the above table.

<div style="background-color: black"><img src="./row_11.png"></div>

If a cell has a borough but a **Not assigned** neighborhood, then the neighborhood will be the same as the borough. So for the **9th** cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be **Queen's Park**.

<div style="background-color: black"><img src="./row_9.png"></div>

In [53]:
df['PostalCode'].unique()

array(['M3A', 'M4A', 'M5A', 'M6A', 'M7A', 'M9A', 'M1B', 'M3B', 'M4B',
       'M5B', 'M6B', 'M9B', 'M1C', 'M3C', 'M4C', 'M5C', 'M6C', 'M9C',
       'M1E', 'M4E', 'M5E', 'M6E', 'M1G', 'M4G', 'M5G', 'M6G', 'M1H',
       'M2H', 'M3H', 'M4H', 'M5H', 'M6H', 'M1J', 'M2J', 'M3J', 'M4J',
       'M5J', 'M6J', 'M1K', 'M2K', 'M3K', 'M4K', 'M5K', 'M6K', 'M1L',
       'M2L', 'M3L', 'M4L', 'M5L', 'M6L', 'M9L', 'M1M', 'M2M', 'M3M',
       'M4M', 'M5M', 'M6M', 'M9M', 'M1N', 'M2N', 'M3N', 'M4N', 'M5N',
       'M6N', 'M9N', 'M1P', 'M2P', 'M4P', 'M5P', 'M6P', 'M9P', 'M1R',
       'M2R', 'M4R', 'M5R', 'M6R', 'M7R', 'M9R', 'M1S', 'M4S', 'M5S',
       'M6S', 'M1T', 'M4T', 'M5T', 'M1V', 'M4V', 'M5V', 'M8V', 'M9V',
       'M1W', 'M4W', 'M5W', 'M8W', 'M9W', 'M1X', 'M4X', 'M5X', 'M8X',
       'M4Y', 'M7Y', 'M8Y', 'M8Z'], dtype=object)

In [73]:
# Reindexing
df.index = range(0, len(df))
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [77]:
# Defining a lamnda function to process each row.
def f(x):
    return pd.Series(dict(A = x.loc[:,'PostalCode'].values[0], 
                        B = x.loc[:,'Borough'].values[0], 
                        C = x.loc[:,'Borough'].values[0] if (x['Neighborhood'].values[0] is np.nan) else ('%s' % ', '.join(x['Neighborhood']))))
    
df_grouped = df.groupby('PostalCode').apply(f)

In [78]:
df_grouped.head()

,A,B,C
PostalCode,,,
M1B,M1B,Scarborough,"Rouge, Malvern"
M1C,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
M1E,M1E,Scarborough,"Guildwood, Morningside, West Hill"
M1G,M1G,Scarborough,Woburn
M1H,M1H,Scarborough,Cedarbrae


In [79]:
# Reindexing
df_grouped.index = range(0, len(df_grouped))
df_grouped.columns = ['PostalCode', 'Borough', 'Neighborhood']
df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [80]:
df_grouped.shape

(103, 3)

==============================================================================================================

## Adding Coordinates to DataFrame

In [82]:
coords = pd.read_csv("Geospatial_Coordinates.csv")
coords.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [155]:
lats = [] 
lons = []
for index, row in df_grouped.iterrows():
    lats.append(coords[coords['Postal Code'] == row['PostalCode']].values[0][1])
    lons.append(coords[coords['Postal Code'] == row['PostalCode']].values[0][2])
    
df_grouped = df_grouped.assign(latitude = lats, longitude = lons)
df_grouped.head()

,PostalCode,Borough,Neighborhood,latitude,longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [162]:
# Working only with Toronto's Boroughs
df_toronto = df_grouped[df_grouped['Borough'].str.contains(r'Toronto')]
df_toronto.head()

,PostalCode,Borough,Neighborhood,latitude,longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [163]:
# one hot encoding
toronto_onehot = pd.get_dummies(df_toronto[['Neighborhood']], prefix="", prefix_sep="")
toronto_onehot

,"Adelaide, King, Richmond",Berczy Park,"Brockton, Exhibition Place, Parkdale Village",Business Reply Mail Processing Centre 969 Eastern,"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara","Cabbagetown, St. James Town",Central Bay Street,"Chinatown, Grange Park, Kensington Market",Christie,Church and Wellesley,...,Roselawn,"Runnymede, Swansea","Ryerson, Garden District",St. James Town,Stn A PO Boxes 25 The Esplanade,Studio District,"The Annex, North Midtown, Yorkville",The Beaches,"The Beaches West, India Bazaar","The Danforth West, Riverdale"
37,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
41,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
42,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
43,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
44,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
45,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
46,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
47,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
48,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
